In [1]:
import pandas as pd
import numpy as np

from textVectorizer import *

In [10]:
f = pd.read_csv("data_vectorized.csv", index_col=0)
f

,abadon,abandon,abat,abb,abbey,abbott,abbrevi,abid,abidjan,abil,...,zone,zong'ai,zoom,zovirax,zuber,zurich,zwetchenbaum,zwetchkenbaum,zx,zyrtec
data,,,,,,,,,,,,,,,,,,,,,
data/C50test/RobinSidel/338537newsML.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data/C50train/JonathanBirt/107612newsML.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data/C50train/TheresePoletti/200003newsML.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data/C50test/JanLopatka/346547newsML.txt,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
data/C50test/TanEeLyn/586394newsML.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
data/C50test/EricAuchard/320619newsML.txt,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
data/C50test/PeterHumphrey/519044newsML.txt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data/C50test/JonathanBirt/462591newsML.txt,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
corp = vectors_from_f(f)
corp

0          Vector(RobinSidel/338537newsML.txt)
1        Vector(JonathanBirt/107612newsML.txt)
2      Vector(TheresePoletti/200003newsML.txt)
3          Vector(JanLopatka/346547newsML.txt)
4            Vector(TanEeLyn/586394newsML.txt)
                        ...                   
245       Vector(EricAuchard/320619newsML.txt)
246     Vector(PeterHumphrey/519044newsML.txt)
247      Vector(JonathanBirt/462591newsML.txt)
248    Vector(KevinDrawbaugh/233324newsML.txt)
249        Vector(JanLopatka/192103newsML.txt)
Name: data, Length: 250, dtype: object

In [12]:
def knn(vec: Vector, corp: pd.Series, k: int, metric: str = "cos") -> str:
    sims = None
    if metric == "cos":
        sims = corp.apply(lambda v: vec.cos_sim(v))
    elif metric == "okapi":
        sims = corp.apply(lambda v: vec.okapi(v))
    authors = corp.apply(lambda v: v.author())
    top_k_dist_ind = np.argpartition(-sims, k)[:k]
    top_k_c = authors.iloc[top_k_dist_ind]
    return top_k_c.mode().sample(frac=1).iloc[0]

In [62]:
knn(corp.loc[10], corp.drop(10), 3)

'KeithWeir'

In [16]:
def knn_predict(corp: pd.Series, k: int, metric: str = "cos") -> pd.DataFrame:
    pred = pd.Series(corp.index).apply(
        lambda row: knn(corp.loc[row], corp.drop(row), k, metric=metric)
    )
    docs = corp.apply(lambda d: d.name())
    obs = corp.apply(lambda d: d.author())
    pred = pd.concat([docs, obs, pred], axis=1)
    pred.columns = ["doc", "obs", "pred"]
    return pred

In [17]:
results = knn_predict(corp, 13)
results

,vector,doc,obs,pred
0,Vector(RobinSidel/338537newsML.txt),338537newsML.txt,RobinSidel,RobinSidel
1,Vector(JonathanBirt/107612newsML.txt),107612newsML.txt,JonathanBirt,JonathanBirt
2,Vector(TheresePoletti/200003newsML.txt),200003newsML.txt,TheresePoletti,GrahamEarnshaw
3,Vector(JanLopatka/346547newsML.txt),346547newsML.txt,JanLopatka,JohnMastrini
4,Vector(TanEeLyn/586394newsML.txt),586394newsML.txt,TanEeLyn,PeterHumphrey
...,...,...,...,...
245,Vector(EricAuchard/320619newsML.txt),320619newsML.txt,EricAuchard,MartinWolk
246,Vector(PeterHumphrey/519044newsML.txt),519044newsML.txt,PeterHumphrey,PeterHumphrey
247,Vector(JonathanBirt/462591newsML.txt),462591newsML.txt,JonathanBirt,JonathanBirt
248,Vector(KevinDrawbaugh/233324newsML.txt),233324newsML.txt,KevinDrawbaugh,MartinWolk


In [18]:
(results['obs'] == results['pred']).sum() / len(results.index)

0.344

In [73]:
okapi_results = knn_predict(corp, 13, "okapi")
okapi_results

,vector,doc,obs,pred
0,Vector(MureDickie/349836newsML.txt),349836newsML.txt,MureDickie,JimGilchrist
1,Vector(JanLopatka/462517newsML.txt),462517newsML.txt,JanLopatka,JimGilchrist
2,Vector(PierreTran/433717newsML.txt),433717newsML.txt,PierreTran,MarcelMichelson
3,Vector(TheresePoletti/310688newsML.txt),310688newsML.txt,TheresePoletti,JimGilchrist
4,Vector(ScottHillis/348803newsML.txt),348803newsML.txt,ScottHillis,PeterHumphrey
...,...,...,...,...
245,Vector(MichaelConnor/453294newsML.txt),453294newsML.txt,MichaelConnor,DarrenSchuettler
246,Vector(SarahDavison/324597newsML.txt),324597newsML.txt,SarahDavison,MarkBendeich
247,Vector(MartinWolk/311937newsML.txt),311937newsML.txt,MartinWolk,MartinWolk
248,Vector(JanLopatka/194352newsML.txt),194352newsML.txt,JanLopatka,MarkBendeich


In [74]:
(okapi_results['obs'] == okapi_results['pred']).sum() / len(okapi_results.index)

0.24

In [19]:
import pandas as pd

In [21]:
pd.read_csv("knn_results.csv", index_col=0).to_csv("knn_results.csv", index=False)